In [1]:
from poker import Range
from poker.hand import Combo
import holdem_calc

import holdem_functions
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
import dask
from dask import delayed
import multiprocessing as mp
import numpy as np

### Optimal bet using Kelly Criterion

In [2]:
def calc_kelly(win_odds, bet_odds):
    kelly_bet = round((bet_odds*win_odds-(1-win_odds))/bet_odds, 3)
    return kelly_bet

### Calculate the poker bet based on your hand, the board, and the villain range

In [104]:
def calc_poker_bet(my_hand, board, bankroll, return_multiplier, min_kelly = .5, max_kelly = 2, villain_range = Range('77+, AT+, KJ+'), 
                   calc_villain_range_on = False, use_dask = True, villain_hand = None, exact_calculation = True,
                   verbose = True, num_sims = 1, read_from_file = None, print_elapsed_time = True):
    
    """
    Output
    First element of odds contains my probability of winning/tie/lose 
    Second element of odds contains my probablities
    Third element of odds contains villain probabilities
    """
    
    # @param my_hand is my hand in the format of Combo('AdKs') - must be 2 cards
    # @param board is a list of cards in the format ['Kd', 'Ah', '2c']
    
    # @param  bankroll is my current bankroll 
    # @param return_multiplier is the kelly return multiplier (e.g. if you win 2:1 then return_multiplier is 2)
    # @params - exact_calculation, num_sims, read_from_file, villain_hand, verbose, print_elapsed_time will stay default
    # @param villain_range is the range of possible hands that the villain has - must be in the format Range('77+, AT+, KJ+')
    
    odds = holdem_calc.calculate_odds_villan(board = board, exact = exact_calculation, 
                                             num = num_sims, input_file = read_from_file, 
                                             hero_cards = my_hand, villan_cards = villain_hand, 
                                             verbose = verbose, print_elapsed_time = print_elapsed_time)
    
    if calc_villain_range_on == True:
        
        def calc_items(villain_combos=villain_range.combos):
    
            items = [holdem_calc.calculate_odds_villan(board = board, exact = exact_calculation, 
                                                       num = num_sims, input_file = read_from_file, 
                                                       hero_cards = my_hand, villan_cards = villain_hand, 
                                                       verbose = verbose, print_elapsed_time = print_elapsed_time) 
                     for villain_hand in villain_combos]
 
            return items
        
        ######### Calc items #########
        
        if use_dask == False:
            
            items = calc_items()
            
        else:
            
            num_cores = mp.cpu_count()
            delayed_list = []
            start_pos = 0
            end_pos = int(np.floor(len(villain_range.combos) / num_cores))
            chunk_len = int(np.floor(len(villain_range.combos) / num_cores))

            for chunk in range(num_cores):
                if chunk != num_cores - 1:
                    delayed_list.append(delayed(calc_items)(villain_range.combos[start_pos:end_pos]))
                    start_pos = start_pos + chunk_len
                    end_pos = end_pos + chunk_len

                else:
                    delayed_list.append(delayed(calc_items)(villain_range.combos[start_pos:]))
            
            items = list(np.concatenate(dask.compute(*delayed_list)))
            
        ######### End Calc items #########
        

        print(display(HTML(villain_range.to_html())))
        
#         for hand_ranking in holdem_functions.hand_rankings:
#             print(hand_ranking +": " + str(np.mean([res[1][1][hand_ranking] for res in items if res])))
        
        hr = [hand_ranking +": " + str(np.mean([res[1][1][hand_ranking] for res in items if res])) 
              for hand_ranking in holdem_functions.hand_rankings]
    
        
        kelly_bet = calc_kelly(win_odds = odds[0]['win'], bet_odds = return_multiplier)
        min_kelly = kelly_bet*min_kelly
        true_kelly = kelly_bet
        max_kelly = kelly_bet*max_kelly
        
        kelly_recommendations = {'min_kelly':min_kelly*bankroll, 
                                 'true_kelly':true_kelly*bankroll, 
                                 'max_kelly':max_kelly*bankroll}
#         return {'kelly_recommendations':kelly_recommendations, 'odds':odds, 'hand_ranking':hr, 'items':items} # items too big
        return {'kelly_recommendations':kelly_recommendations, 'odds':odds, 'hand_ranking':hr}
    
    else:    
        
        kelly_bet = calc_kelly(win_odds = odds[0]['win'], bet_odds = return_multiplier)
        min_kelly = kelly_bet*min_kelly
        true_kelly = kelly_bet
        max_kelly = kelly_bet*max_kelly
        
        kelly_recommendations = {'min_kelly':min_kelly*bankroll, 
                                 'true_kelly':true_kelly*bankroll,
                                 'max_kelly':max_kelly*bankroll}

        
        return {'kelly_recommendations':kelly_recommendations, 'odds':odds, 'hand_ranking':hr}

In [105]:
tmp = calc_poker_bet(Combo('5s8h'), board = ['Jd', '2h', '3s'], bankroll = 7.5, use_dask = False,
                     return_multiplier=2, calc_villain_range_on = True)

Time elapsed:  13.705389261245728
Time elapsed:  0.012009620666503906
Time elapsed:  0.015942811965942383
Time elapsed:  0.012019634246826172
Time elapsed:  0.004038333892822266
Time elapsed:  0.01565694808959961
Time elapsed:  0.0
Time elapsed:  0.015587091445922852
Time elapsed:  0.020624876022338867
Time elapsed:  0.0
Time elapsed:  0.015623092651367188
Time elapsed:  0.0
Time elapsed:  0.02871990203857422
Time elapsed:  0.01997232437133789
Time elapsed:  0.015977144241333008
Time elapsed:  0.01205754280090332
Time elapsed:  0.007508039474487305
Time elapsed:  0.0156252384185791
Time elapsed:  0.0
Time elapsed:  0.02148747444152832
Time elapsed:  0.023966073989868164
Time elapsed:  0.015977144241333008
Time elapsed:  0.012052774429321289
Time elapsed:  0.011040449142456055
Time elapsed:  0.017186403274536133
Time elapsed:  0.02398514747619629
Time elapsed:  0.015620946884155273
Time elapsed:  0.0
Time elapsed:  0.015620708465576172
Time elapsed:  0.015621185302734375
Time elapsed:  

AA,AKs,AQs,AJs,ATs,,,,,,,,
AKo,KK,KQs,KJs,,,,,,,,,
AQo,KQo,QQ,,,,,,,,,,
AJo,KJo,,JJ,,,,,,,,,
ATo,,,,TT,,,,,,,,
,,,,,99,,,,,,,
,,,,,,88,,,,,,
,,,,,,,77,,,,,
,,,,,,,,,,,,
,,,,,,,,,,,,
,,,,,,,,,,,,


None


In [106]:
tmp

{'kelly_recommendations': {'min_kelly': -0.6075,
  'true_kelly': -1.215,
  'max_kelly': -2.43},
 'odds': [{'tie': 0.02094301011969837,
   'win': 0.22536558928788347,
   'lose': 0.7536914005924181},
  [{'High Card': 0.38482886216466233,
    'Pair': 0.4884366327474561,
    'Two Pair': 0.08325624421831637,
    'Three of a Kind': 0.013876040703052728,
    'Straight': 0.02960222016651249,
    'Flush': 0.0,
    'Full House': 0.0,
    'Four of a Kind': 0.0,
    'Straight Flush': 0.0,
    'Royal Flush': 0.0},
   {'High Card': 0.2227567067530065,
    'Pair': 0.4771788187144339,
    'Two Pair': 0.21177361029349928,
    'Three of a Kind': 0.03923976116390548,
    'Straight': 0.02524037787682561,
    'Flush': 0.006441846774871752,
    'Full House': 0.016550332184004708,
    'Four of a Kind': 0.0007849073528999524,
    'Straight Flush': 2.8032405460712584e-05,
    'Royal Flush': 5.606481092142517e-06}]],
 'hand_ranking': ['High Card: 0.178997668997669',
  'Pair: 0.49259518259518265',
  'Two Pair: 0